In this exercise, you will create a command line interactive movie recommender system. Your program will ask the user to rate ten movies and recommend other movies they might like by finding users with similar tastes in movies and recommending movies they rated highly. This technique is often called collaborative filtering.

You can find the data for this exercise in the movielens folder. You will need the movies.csv and ratings.csv files to complete this assignment.

**1. Recommendation Engine**

The first step in the process is creating a matrix of user ratings for each movie. If there are users and movies, then the user rating matrix will be N x M.

<img src="files/Images/1.jpg">

The recommendation engine should take as input user ratings one or more movies and output an ordered list of movie recommendations. Test your recommendation engine by selecting and rating ten movies from the top 200 most rated movies. The following is an example of test input.

<img src="files/Images/2.jpg">

In order to determine which movies to recommend, you should first compute the cosine distance of each existing user’s ratings from the inputs given. You can compute the cosine distance between two vectors using scipy or scikit-learn. Next, compute a weighted average of the ratings of all user ratings using the cosine distance as the weight. In other words, users who gave similar ratings to the input ratings, are weighted higher than those who gave less similar ratings.

In mathematical terms, if there are N movies and M users, the user rating matrix is an N x M matrix. The user input is a vector of length N. One row of the user rating matrix represents the movie ratings for one user. If you compute the cosine similarity between that row and the user’s input, you have a measure of how close that user’s ratings are to the input ratings. If the other user rating is everything opposite, the cosine similarity is -1. If they rate everything the same, the cosine similarity is 1.

The following Python code demonstrates how to calculate the movie recommendations without using Numpy’s matrix math operations. It uses Python list comprehensions and for loops for the purpose of clarity. The output of this function is a list of length N (the number of unique movies) with a suggested rating for each movie.

<img src="files/Images/3.jpg">

This function does not take into account movies the user has already rated, so you should make sure you remove those movies from your recommendations. Additionally, an optimal solution would use Numpy’s matrix mathematics, rather than Python lists.

**2. Movie Search Engine**

Now that you have a recommendation engine, you need to provide a way for users to find movies to rate. You will need to create a function that takes in a search parameter and returns a ranked list of movies that best matches the input. For this data set, there are less than 10,000 movies and you only need to worry about searching the titles for those movies. Therefore, we do not need to worry as much about coming up with an optimal solution that scales for larger datasets.

When returning candidate movie titles, you will want to return the titles that match the search input with the highest probability. Consider dividing up the titles and the user input into n-grams, but instead of using n-grams of works, the n-grams are characters in the string.

For example, the title Batman contains the bigrams, [‘ba’, ‘tm’, ‘an’, ‘at’, ‘ma’]. You could then match that input title to titles that contain those bigrams with the highest probability. Find a search method that generally returns correct recommendations based off the search input.

**3. Movie Recommendation Application**

In this part, you create an interactive movie recommendation application by combining the movie recommendation engine and the movie search engine. To accomplish this, you will create a simple command line application. Upon starting the application, you should ask the user to find a movie to rate. It should return a list of numbered movies or an “I don’t see what I’m looking for” option.

<img src="files/Images/4.jpg">

If the user selects a movie, they need to enter a rating from zero to five and that movie will be added to their list of movies.

<img src="files/Images/5.jpg">

After the user has rated at least five movies, give them the option to rate more movies or to get recommendations. Your application should return a ranked list of five movie recommendations.

<img src="files/Images/6.jpg">

### 1. Recommendation Engine

In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from scipy.sparse.linalg import svds
from collections import Counter
from nltk.util import ngrams
from nltk.corpus import stopwords
import string
import re
import nltk

# Create stop words

stop_words = stopwords.words('english')

# Setting this option off since I need to update a dataframe in a loop

pd.options.mode.chained_assignment = None

In [2]:
def read_csv_file(file):
    """
    Read data from csv file and return a 
    pandas data frame
    Args: file name
    Returns: pandas data farme
    """
    df = []
    df = pd.read_csv(file)

    return df

In [3]:
# Read the movies and ratings data

movies_df = read_csv_file('data/movielens/movies.csv')
ratings_df = read_csv_file('data/movielens/ratings.csv')

print('Movies:', movies_df.head())
print('Ratings:', ratings_df.head())

Movies:    movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  
Ratings:    userId  movieId  rating   timestamp
0       1       31     2.5  1260759144
1       1     1029     3.0  1260759179
2       1     1061     3.0  1260759182
3       1     1129     2.0  1260759185
4       1     1172     4.0  1260759205


In [4]:
# Format the ratings matrix by transposing

R_df = ratings_df.pivot(index = 'userId', columns ='movieId', values = 'rating').fillna(0)
R_df.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
# De-mean the data

R = R_df.values
user_ratings_mean = np.mean(R, axis = 1)
R_demeaned = R - user_ratings_mean.reshape(-1, 1)

# Define latent factors to use to approximate the original ratings matrix 

U, sigma, Vt = svds(R_demeaned, k = 50)

# Diagonal matrix form

sigma = np.diag(sigma)

# Predictions from the Decomposed Matrices

all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)
preds_df = pd.DataFrame(all_user_predicted_ratings, columns = R_df.columns)

In [6]:
def recommend_movies(predictions_df, userID, movies_df, original_ratings_df, num_recommendations=5):
    """
    Recommendation function
    Args: prediction dataframe, user id, all movies data frame, 
          original ratings dataframe, number of recommendations (dafault = 5)
    Returns: prints number of movies already rated by the user and returns 
             the recommended movies dataframe
    """
    
    # Get and sort the user's predictions
    
    user_row_number = userID - 1 # UserID starts at 1, not 0
    sorted_user_predictions = predictions_df.iloc[user_row_number].sort_values(ascending=False)
    
    # Get the user's data and merge in the movie information.
    
    user_data = original_ratings_df[original_ratings_df['userId'] == (userID)]
    user_full = (user_data.merge(movies_df, how = 'left', left_on = 'movieId', right_on = 'movieId').
                     sort_values(['rating'], ascending=False)
                 )

    print('User {0} has already rated {1} movies.'.format(userID, user_full.shape[0]))
    print('Recommending the highest {0} predicted ratings movies not already rated.'.format(num_recommendations))
    
    # Recommend the highest predicted rating movies that the user hasn't seen yet.
    
    recommendations = (movies_df[~movies_df['movieId'].isin(user_full['movieId'])].
         merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
               left_on = 'movieId',
               right_on = 'movieId').
         rename(columns = {user_row_number: 'Predictions'}).
         sort_values('Predictions', ascending = False).
                       iloc[:num_recommendations, :-1]
                      )

    return user_full, recommendations

# Excute function

already_rated, predictions = recommend_movies(preds_df, 100, movies_df, ratings_df, 10)
predictions

User 100 has already rated 25 movies.
Recommending the highest 10 predicted ratings movies not already rated.


,movieId,title,genres
428,494,Executive Decision (1996),Action|Adventure|Thriller
652,832,Ransom (1996),Crime|Thriller
219,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi
28,36,Dead Man Walking (1995),Crime|Drama
638,805,"Time to Kill, A (1996)",Drama|Thriller
630,788,"Nutty Professor, The (1996)",Comedy|Fantasy|Romance|Sci-Fi
327,376,"River Wild, The (1994)",Action|Thriller
549,653,Dragonheart (1996),Action|Adventure|Fantasy
670,852,Tin Cup (1996),Comedy|Drama|Romance
12,17,Sense and Sensibility (1995),Drama|Romance


### 2. Movie Search Engine

In [7]:
def process_text(text):
    """
    Remove year of release, punctuation and 
    stop words from string (movie names)
    Args: text string
    Returns: lower case text string (removed punctuation, stop words, space)
    """
    
    # Create empty string to store output
    
    textcln = ''
    
    # Replace all whitespaces and convert to lower case
    
    pattern = re.compile(r'\s+')
    textcln = re.sub(pattern, '', text).lower()
    
    # If the movie name ends with a ')', remove the year at the end
    
    if text.strip().endswith(')'):
        textcln = textcln[:-6]
    else:
        textcln = textcln
    
    # Remove punctuation and stop words
    textcln = textcln.translate(str.maketrans('', '', string.punctuation))
    textcln = ' '.join([word for word in textcln.split() if word not in stop_words])  
    
    if textcln.strip().endswith(')'):
        textcln = ''.join(textcln.split())[:-6].upper()
    
    return textcln


def jaccard_distance(set1, set2):
    """
    Calculate Jaccard distance between two sets
    Args: set 1 and set 2
    Returns: Jaccard distance between set 1 and set 2
    """
    
    set1_unq = set(set1)
    set2_unq = set(set2)
    num_both = len(set1_unq.intersection(set2_unq))
    num_total = len(set1_unq.union(set2_unq))
    
    return num_both/num_total


def get_bigrams(text, isClean = 'Y'):
    """
    Generate N grams by taking text, x of N gram 
    and return the desired top X desired items
    Args: text string, cleaning flag Y/N
    Returns: list of bigrams
    """
    
    if isClean == 'Y':
        text = process_text(text)
        
    # Generate bigrams out of the new spaced string
    
    spaced = ''
    for ch in text:
        spaced = spaced + ch + ' '

    
    tokenized = spaced.split(" ")
    myList = list(nltk.bigrams(tokenized))
       
    # Join the items in each tuple in myList together and put them in a new list

    Bigrams = []

    for i in myList:
        Bigrams.append((''.join([w + ' ' for w in i])).strip())

    return Bigrams


def search_movie(search_movie, all_movies, N=10):
    """
    Returns matching movies (by name) following a basic algorithm
    Args: string of movie name to search, all movies dataframe, 
          number of similar movies (defaulted to 10)
    Returns: dataframe of movies matching with search, sorted by similarity
    """

    # Get the bigram of the searched movie
    
    source_ngram = get_bigrams(search_movie)
    
    # Copy the movie database and add a similarity score column
    
    match_df = movies_df
    match_df['similarity'] = 0
    
    # Get bigrams of target movie and calculate similarity score
    # by comapring the occurances of bigrams between source and target

    for i in range(0, len(match_df)):
        target_name = match_df['title'].iloc[i]
        
        if process_text(search_movie) == process_text(target_name):
            continue
            
        target_ngram = get_bigrams(target_name)
        
        # I am not using this mathing algorithm anymore
        #match = len(set(source_ngram) & set(target_ngram))
        
        # Using jaccard distance to match similarity
        
        match = jaccard_distance(source_ngram,target_ngram)        
        match_df['similarity'].iloc[i] = match

    # Remove rows that did not match
        
    match_df = match_df[match_df.similarity != 0]
    
    return match_df.sort_values('similarity', ascending=False).head(N)
    
    
search_movie('Bells Are Ringing (1960)',movies_df, 10)

,movieId,title,genres,similarity
7515,77201,Valhalla Rising (2009),Action|Drama|War,0.300000
307,342,Muriel's Wedding (1994),Comedy,0.285714
826,1022,Cinderella (1950),Animation|Children|Fantasy|Musical|Romance,0.277778
1046,1292,Being There (1979),Comedy|Drama,0.277778
7079,63239,Cinderella (1997),Children|Fantasy|Musical|Romance,0.277778
5161,7454,Van Helsing (2004),Action|Adventure|Fantasy|Horror,0.277778
8845,130073,Cinderella (2015),Children|Drama|Fantasy|Romance,0.277778
6251,38499,Angels in America (2003),Drama|Fantasy,0.272727
4534,6242,Ringu (Ring) (1998),Horror|Mystery|Thriller,0.266667
8536,109487,Interstellar (2014),Sci-Fi|IMAX,0.263158


### 3. Movie Recommendation Application

In [53]:
def take_movie(txt,moviecount):
    """
    Take a movie search string and return matches
    Args: movie name search string, number of matches
    Returns: data frame with matching movies
    """
        
    result = search_movie(txt,movies_df, moviecount)['title']

    for i in range(1):
        result.loc[i] = 'I do not see what I am looking for'
    
    result_df = result.to_frame()
    result_df.insert(0, 'ID', range(1, 1 + len(result_df)))

    return result_df


def take_ratings(selection, moviecount, input_df):
    """
    Accepts rating to a movie
    Args: movie serieal number, number of movies, 
          dataframe with list of movies
    Returns: rating as input by user
    """
    
    try:        
        if(selection >= 1 and selection <= moviecount):
            if(selection == moviecount+1):
                print('Exiting...')
            else:
                selection_df = input_df[(input_df.ID == selection)]
                selection_title = input_df[(input_df.ID == selection)]['title'].to_string()   
                print('How much did you enjoy ' +selection_title+ ' on a scale of 0 to 5?')
                rating = input('(0 - Hated it, 5 - Loved it):')
                try:
                    val1 = int(rating)
                    if(val1 >= 0 and val1 <= 5):
                        print('Ratings saved.')
                        return val1
                    else:
                        print('Please enter a value between 0 and 5')
                except ValueError:
                    print('Please enter ratings in integer')            
        else:
            print("Please enter a valid id")
    except ValueError:
        print('Please enter integer')
        
        
def take_input():
    """
    Start session of an user with user id
    Args: user id, movie serial number
    Returns: after 5 inputs
    """
    
    user = input('Please enter your user id: ')
    txt = input('Find a movie to rate: ')
    moviecount = input('How many results do you want? ')
    moviecount = int(moviecount)
    input_df = take_movie(txt,moviecount)
    print(input_df)
    
    for i in range(1,6):        
        selection = input('Enter ID of the movie you want to select: ')
        selection = int(selection)
        rating = take_ratings(selection, moviecount, input_df)
        
        # Due to time constrain I am writing the remaining algorithm
        # 1. Get the movie index, user id and ratings
        # 2. Add the information to the original ratings data frame
        # 3. Ask for new recommendation and call this function recursively
        
# Run logic
take_input()

Please enter your user id: sadfaf
Find a movie to rate: sdfdv asfdsf
How many results do you want? 9
      ID                               title
6324   1                     Casanova (2005)
1232   2                  Brassed Off (1996)
2676   3                  Blood Feast (1963)
969    4                   Goodfellas (1990)
3858   5                  Vanilla Sky (2001)
6761   6                Invasion, The (2007)
4617   7                 Nevada Smith (1966)
6355   8  World's Fastest Indian, The (2005)
3838   9              Invasion U.S.A. (1985)
0     10  I do not see what I am looking for
Select a movie and enter its ID: 1
How much did you enjoy 6324    Casanova (2005) on a scale of 0 to 5?
(0 - Hated it, 5 - Loved it):2
Ratings saved.
Select a movie and enter its ID: 3
How much did you enjoy 2676    Blood Feast (1963) on a scale of 0 to 5?
(0 - Hated it, 5 - Loved it):4
Ratings saved.
Select a movie and enter its ID: 5
How much did you enjoy 3858    Vanilla Sky (2001) on a scale of 0 

**End of code**